# Homework 3

## Description

### Data
[Reuters-21578 Text Categorization Collection Data Set](https://archive.ics.uci.edu/ml/datasets/reuters-21578+text+categorization+collection) - about 2 million instances, 28MB in size


News: 21 SGML files
We only deal with news contents inside <body> </body> tags
The other files will not be needed in this homework

### Format
One text file consisting of lines of records.

Each record contains 9 attributes separated by semicolons: 


Reuters-21578, Distribution 1.0 includes five files (all-exchanges-strings.lc.txt, all-orgs-strings.lc.txt, all-people-strings.lc.txt, all-places-strings.lc.txt, and all-topics-strings.lc.txt) which list the names of *all* legal categories in each set. A sixth file, cat-descriptions_120396.txt gives some additional information on the category sets.


### Task
4 subtasks:
+ (30pt) Given the Reuters-21578 dataset, please calculate all k-shingles and output the set representation of the text dataset as a matrix.
+ (30pt) Given the set representation, compute the minhash signatures of all documents using MapReduce.
+ (40pt) Implement the LSH algorithm by MapReduce and output the resulting candidate pairs of similar documents. 
+ (Bonus) Implement K-nearest neighbor (KNN) search using LSH and compare its performance with linear search.

## Output Format

1. Set representation:
A MxN matrix: with rows as shingles and columns as documents (N=21,578) 
2. minhash signatures:
The HxN signature matrix: with H as the number of hash functions, N=21,578
3. candidate pairs:
For each document i, there should be a list of those documents j>i with which i needs to be compared
4. comparison of KNN search an linear search


## Implementation Notes

Note the differences in rows and columns for the input data and the output matrices
* Input: rows as documents
* Output: columns as documents

Your program should be able to accept some parameters:
* k in k-shingles
* Number of hash functions H




## Implementation

In [1]:
import org.apache.spark.sql.SparkSession
import org.apache.spark.SparkContext
import org.apache.spark.SparkConf
import org.apache.hadoop.fs._
import org.apache.hadoop.conf.Configuration

import java.io.{File,PrintWriter}

  def printSample(writer: Any, data: Any, title: String = "", format: String = ""){
    println("\n"+title+" Data Sample: " + format)
    println(data+"\n")
  }

  def printSpark(writer: Any, spark: SparkSession): Unit = {
    println("Spark Entity:       " + spark)
    println("Spark version:      " + spark.version)
    println("Spark master:       " + spark.sparkContext.master)
    println("Running 'locally'?: " + spark.sparkContext.isLocal)
    println("")
  }

  def outputWriter(fileString: String): PrintWriter ={
    val outputPath = new Path(fileString)
    val outputStream = outputPath.getFileSystem(new Configuration()).create(outputPath);
    new PrintWriter(outputStream)
  }

  def getFile(fileString: String): Array[String] ={
    val inputPath = new Path(fileString)
    val inputBuffer = scala.collection.mutable.ArrayBuffer.empty[String]
    val iterator = inputPath.getFileSystem(new Configuration()).listFiles(inputPath, false)
    while(iterator.hasNext()){
        val fileStatus = iterator.next()
        if(fileStatus.isFile()){
          inputBuffer += fileStatus.getPath().toString()
        }
    }
    inputBuffer.toArray
  }


val writer = null
printSpark(writer, spark)

Spark Entity:       org.apache.spark.sql.SparkSession@3bac3c23
Spark version:      2.2.0
Spark master:       local[*]
Running 'locally'?: true



printSample: (writer: Any, data: Any, title: String, format: String)Unit
printSpark: (writer: Any, spark: org.apache.spark.sql.SparkSession)Unit
outputWriter: (fileString: String)java.io.PrintWriter
getFile: (fileString: String)Array[String]
writer: Null = null


In [2]:
val spark2 = spark

var data = getFile("./data")
var sample_data = Array("file:/home/micky/big_data/hw3/data/reut2-017.sgm")

spark2 = org.apache.spark.sql.SparkSession@3bac3c23
data = Array(file:/home/micky/big_data/hw3/data/reut2-017.sgm, file:/home/micky/big_data/hw3/data/reut2-001.sgm, file:/home/micky/big_data/hw3/data/reut2-021.sgm, file:/home/micky/big_data/hw3/data/reut2-003.sgm, file:/home/micky/big_data/hw3/data/reut2-010.sgm, file:/home/micky/big_data/hw3/data/reut2-013.sgm, file:/home/micky/big_data/hw3/data/reut2-004.sgm, file:/home/micky/big_data/hw3/data/reut2-020.sgm, file:/home/micky/big_data/hw3/data/reut2-008.sgm, file:/home/micky/big_data/hw3/data/reut2-014.sgm, file:/home/micky/big_data/hw3/data/reut2-007.sgm, file:/home/micky/big_data/hw3/data/reut2-018.sgm, file:/home/micky/big_data/hw3/data/reut2-005.sgm, file:/home/micky/big_data/hw3/da...


[file:/home/micky/big_data/hw3/data/reut2-017.sgm, file:/home/micky/big_data/hw3/data/reut2-001.sgm, file:/home/micky/big_data/hw3/data/reut2-021.sgm, file:/home/micky/big_data/hw3/data/reut2-003.sgm, file:/home/micky/big_data/hw3/data/reut2-010.sgm, file:/home/micky/big_data/hw3/data/reut2-013.sgm, file:/home/micky/big_data/hw3/data/reut2-004.sgm, file:/home/micky/big_data/hw3/data/reut2-020.sgm, file:/home/micky/big_data/hw3/data/reut2-008.sgm, file:/home/micky/big_data/hw3/data/reut2-014.sgm, file:/home/micky/big_data/hw3/data/reut2-007.sgm, file:/home/micky/big_data/hw3/data/reut2-018.sgm, file:/home/micky/big_data/hw3/data/reut2-005.sgm, file:/home/micky/big_data/hw3/data/reut2-015.sgm, file:/home/micky/big_data/hw3/data/reut2-011.sgm, file:/home/micky/big_data/hw3/data/reut2-002.sgm, file:/home/micky/big_data/hw3/data/reut2-012.sgm, file:/home/micky/big_data/hw3/data/reut2-019.sgm, file:/home/micky/big_data/hw3/data/reut2-009.sgm, file:/home/micky/big_data/hw3/data/reut2-016.sgm,

In [3]:
import scala.collection.mutable.ListBuffer

//TODO: Sample -> Actual
var regex = """(?s)<REUTERS.*?NEWID="(\d*)".*?>.*?<TEXT.*?>(.*?)<\/TEXT>.*?<\/REUTERS>""".r
var body_regex = """(?s).*?<BODY.*?>(.*?)<\/BODY>.*?""".r
var newsParse = spark.sparkContext.emptyRDD[(Int, String)]

sample_data.map{
    path  =>
    val singleNewsParse = spark.sparkContext.wholeTextFiles(path).flatMap{
        case (_, string) =>
        val list =  ListBuffer[Tuple2[Int, String]]()
        for(m <- regex.findAllIn(string).matchData){
            for( n <- body_regex.findAllIn(m.group(2)).matchData){
                list += (Tuple2(m.group(1).toInt, n.group(1).replace("\n Reuter\n&#3;","")))
            }
        }
        list.toSeq
    }
    newsParse = newsParse.union(singleNewsParse)
}
var newsCount = newsParse.count()
print("\nCount: "+ newsCount+"\n")
printSample(writer, newsParse.take(2).mkString("\n\n"), "Parsed Sample")


Count: 940

Parsed Sample Data Sample: 
(17001,The Brazilian Coffee Institute, IBC,
plans to sell in a series of auctions over the next few weeks
robusta coffee purchased in London last year, but details of
where and when auctions will take place are still to be
finalised, IBC president Jorio Dauster told reporters.
    The sales of 630,000 bags of robusta and an unspecified
amount of Brazilian arabica coffee will take place over a
minimum of six months but it is not decided where sales will
take place or whether they will be held weekly or monthly.
    The amount offered at each sale has also not been set, but
could be in the order of 100,000 bags, Dauster said.)

(17002,AMCA International Ltd said it
appointed president and chief executive officer WIlliam Holland
to succeed Kenneth Barclay as chairman.
    Barclay, who is 60 years old, decided not to stand for
reappointment as chairman this year but will continue as a
director, AMCA said.)



regex = (?s)<REUTERS.*?NEWID="(\d*)".*?>.*?<TEXT.*?>(.*?)<\/TEXT>.*?<\/REUTERS>
body_regex = (?s).*?<BODY.*?>(.*?)<\/BODY>.*?
newsParse = UnionRDD[4] at union at <console>:59
newsCount = 940


940

### Task 1 - Calculate K-Shingle Matrix

In [4]:
def wordPreProcess(input: Any): Array[String] = {
        var matchRegex = """([$]?(?:[\w]+(?:[\w',]*[\w]+)+|[\w]))""".r
        var list =  ListBuffer[String]()
        for(m <- matchRegex.findAllIn(input.toString.toLowerCase).matchData;
          e <- m.subgroups)
          list+=e
        list.toArray
    }

wordPreProcess: (input: Any)Array[String]


In [5]:
val kShingleValue = 3
val parseSentenceRegex = """([$]?(?:[\w]+(?:[\w',]*[\w]+)+|[\w]))""".r

var flattenShingles = newsParse.flatMap{
    case (id, string)  =>
    wordPreProcess(string).sliding(kShingleValue).map{
        wordVector =>
        (wordVector.mkString(" "), id)
    }  
}
printSample(writer, flattenShingles.take(20).mkString("\n"), "Parsed Shingle Sample")


Parsed Shingle Sample Data Sample: 
(the brazilian coffee,17001)
(brazilian coffee institute,17001)
(coffee institute ibc,17001)
(institute ibc plans,17001)
(ibc plans to,17001)
(plans to sell,17001)
(to sell in,17001)
(sell in a,17001)
(in a series,17001)
(a series of,17001)
(series of auctions,17001)
(of auctions over,17001)
(auctions over the,17001)
(over the next,17001)
(the next few,17001)
(next few weeks,17001)
(few weeks robusta,17001)
(weeks robusta coffee,17001)
(robusta coffee purchased,17001)
(coffee purchased in,17001)



kShingleValue = 3
parseSentenceRegex = ([$]?(?:[\w]+(?:[\w',]*[\w]+)+|[\w]))
flattenShingles = MapPartitionsRDD[5] at flatMap at <console>:46


MapPartitionsRDD[5] at flatMap at <console>:46

In [6]:
var matrixIndex = spark.sparkContext.broadcast(newsParse.map{
    case(id, string) =>
    id.toInt
}.sortBy{
    case (id) => id
}.zipWithIndex.collectAsMap())

printSample(writer, matrixIndex.value.take(5).mkString("\n"), "Ziped With Index Sample")


Ziped With Index Sample Data Sample: 
17758 -> 719
17417 -> 411
17408 -> 402
17004 -> 3
17749 -> 711



matrixIndex = Broadcast(6)


Broadcast(6)

In [7]:
var groupedShingles = flattenShingles.groupByKey()

printSample(writer, groupedShingles.take(5).mkString("\n"), "Grouped Shingle Sample")


Grouped Shingle Sample Data Sample: 
(attacks on the,CompactBuffer(17406))
(he said and,CompactBuffer(17166, 17760, 17915))
(more than four,CompactBuffer(17422, 17644, 17871))
(improving high quality,CompactBuffer(17474))
(expire at end,CompactBuffer(17043))



groupedShingles = ShuffledRDD[11] at groupByKey at <console>:46


ShuffledRDD[11] at groupByKey at <console>:46

In [8]:
var newsCount = spark.sparkContext.broadcast(newsParse.count())
var shinglesCount = spark.sparkContext.broadcast(groupedShingles.count())
print("Count: "+shinglesCount.value+"\n")

var shingleMatrix = groupedShingles.map{
    case (text, iterable) =>
    var row = Array.fill(newsCount.value.toInt){false}
    iterable.toVector.map{
        id =>
        row(matrixIndex.value(id).toInt) = true
    }
    row
}

Count: 128427


newsCount = Broadcast(10)
shinglesCount = Broadcast(12)
shingleMatrix = MapPartitionsRDD[12] at map at <console>:52


MapPartitionsRDD[12] at map at <console>:52

In [ ]:
val shingleString = shingleMatrix.map{
      array =>
      array.map{bool => if(bool) '1' else '0'}.mkString(",")
}

shingleString.saveAsTextFile("./output/shingleMatrix")

###  Task 2 - Compute Min-Hash Signatures

In [9]:
var numOfHashFunctions = spark.sparkContext.broadcast(100)
var prime = 21601

val rand = scala.util.Random
var hashFunctionMatrix = Array.fill(numOfHashFunctions.value,2){rand.nextInt(2000) - 1000}

numOfHashFunctions = Broadcast(13)
prime = 21601
rand = scala.util.Random$@20b2fb24
hashFunctionMatrix = Array(Array(466, -647), Array(-167, -972), Array(-36, -429), Array(-328, 8), Array(49, -884), Array(737, -938), Array(-813, 808), Array(439, 751), Array(705, -694), Array(115, 8), Array(-999, 532), Array(719, -749), Array(-649, -218), Array(-131, -666), Array(604, -933), Array(257, -934), Array(890, 108), Array(-896, -815), Array(540, 751), Array(-768, 821), Array(994, 448), Array(126, -247), Array(-103, 602), Array(591, -321), Array(297, -687), Array(206, 310), Array(233, -289), Array(500, -548), Array(733, 826), Array(-938, 952), Array(271, 175), Array(24, -787), Array(-4, -755), Array(944, -452), ...


[[466, -647], [-167, -972], [-36, -429], [-328, 8], [49, -884], [737, -938], [-813, 808], [439, 751], [705, -694], [115, 8], [-999, 532], [719, -749], [-649, -218], [-131, -666], [604, -933], [257, -934], [890, 108], [-896, -815], [540, 751], [-768, 821], [994, 448], [126, -247], [-103, 602], [591, -321], [297, -687], [206, 310], [233, -289], [500, -548], [733, 826], [-938, 952], [271, 175], [24, -787], [-4, -755], [944, -452], [-203, 256], [517, 985], [597, -728], [-779, -169], [-381, 277], [518, 867], [-51, 481], [-524, 794], [548, 123], [937, -632], [-745, 811], [75, 14], [200, -217], [688, -801], [746, 753], [60, -969], [956, -470], [11, 57], [-561, -776], [-934, 915], [51, 559], [-273, -176], [-608, 398], [-439, -486], [144, -343], [849, 409], [506, -769], [-224, -115], [-519, -217], [91, 932], [555, 892], [-214, 520], [549, -462], [838, 440], [-892, -685], [-992, 225], [846, -37], [-34, 713], [-393, 364], [768, -40], [768, -209], [-383, 836], [-247, 120], [-679, 999], [-249, 804]

In [10]:
def hashFunction(input: Int): Array[Int] = {
//     var hashResultMatrix = Array.ofDim[String](numOfHashFunctions)
    hashFunctionMatrix.map{
        array =>
        (Math.abs(array(0).toInt * input + array(1).toInt)%prime)%newsCount.value.toInt
    }
}

hashFunction: (input: Int)Array[Int]


In [ ]:
val signatureMatrix = shingleMatrix.zipWithIndex.mapPartitions{
    itr =>
    
    itr.map{
        case(rowArray, index) =>
        var matrix = Array.fill(numOfHashFunctions.value,newsCount.value.toInt){99999}
        var hashResult = hashFunction(index.toInt).toSeq
        rowArray.zipWithIndex.map{case (x,i) =>
            if(x!=false){
                for(n <- 0 until numOfHashFunctions.value){
                    matrix(n)(i) = hashResult(n)
                }
            }
        }      
        matrix
    }

}.reduce{
    case (matrix1, matrix2) =>
    for(y <- 0 until matrix1.length;
      x <- 0 until matrix1(y).length){
        if(matrix1(y)(x) > matrix2(y)(x)){
            matrix1(y)(x) = matrix2(y)(x)
        }
    }
    matrix1
}

In [ ]:
val signatureString = signatureMatrix.map{
      array =>
      array.mkString(",")
}.mkString("\n")

val local_writer =  outputWriter("./output/signatureMatrix")
try {local_writer.write(signatureString)} finally {local_writer.close()}

In [ ]:
val hashFunctionString = hashFunctionMatrix.map{
      array =>
      array.mkString(",")
}.mkString("\n")

val local_writer =  outputWriter("./output/hashFunctionMatrix")
try {local_writer.write(hashFunctionString)} finally {local_writer.close()}

### Task 3 - Implement LSH

In [12]:
var rowsPerBand = spark.sparkContext.broadcast(5)
var jaccardSimilarity = spark.sparkContext.broadcast(0.8)
var bandMax = Math.ceil(numOfHashFunctions.value/rowsPerBand.value)
var bandIndexMap = spark.sparkContext.parallelize((0 until bandMax.toInt).toList)
var signatureBroadcast = spark.sparkContext.broadcast(signatureMatrix)

rowsPerBand = Broadcast(15)
jaccardSimilarity = Broadcast(16)
bandMax = 20.0
bandIndexMap = ParallelCollectionRDD[15] at parallelize at <console>:62
signatureBroadcast = Broadcast(17)


Broadcast(17)

In [ ]:
var LSH_matrix = bandIndexMap.map{
    n =>
    var matrix = Array.fill(newsCount.value.toInt,newsCount.value.toInt){false}
    for(y <- 0 until newsCount.value.toInt;
      x <- (y + 1) until newsCount.value.toInt){
        var matchCounter = 0;
        for(count <- 0 until rowsPerBand.value.toInt){
            if(signatureBroadcast.value(n * rowsPerBand.value.toInt + count)(y) == signatureBroadcast.value(n * rowsPerBand.value.toInt + count)(x)){
                matchCounter += 1;
            }
        }
        if(matchCounter.toDouble/rowsPerBand.value >= jaccardSimilarity.value){
            matrix(y)(x) = true;
            matrix(x)(y) = true;
        }
    }
    matrix
}.reduce{
    case (matrix1, matrix2) =>
    for(y <- 0 until matrix1.length;
      x <- 0 until matrix1(y).length){
        if(matrix2(y)(x)){
            matrix1(y)(x) = true
        }
    }
    matrix1
}

LSH_matrix(0).take(10).foreach(println)

In [ ]:
var LSH_pairs = LSH_matrix.map{
    array =>
    var list =  ListBuffer[Int]()
    array.zipWithIndex.map{
        case (v, i) =>
        if(v==true){
            list+=i
        }        
    }
    list.toSeq
}
println(LSH_pairs(9))

In [ ]:
val LSH_String = LSH_pairs.zipWithIndex.map{
      case(array, i) =>
      (i+1) + ": " + array.mkString(",")
}.mkString("\n")

val local_writer =  outputWriter("./output/LSH_Pairs")
try {local_writer.write(LSH_String)} finally {local_writer.close()}

### Task 4 - Comparing LSH vs Linear Search Speed

In [ ]:
val groupedFlattenShingles = flattenShingles.map{
    case (shingle, id) =>
    ( matrixIndex.value(id).toInt, shingle)
}.groupByKey()

val mapFlattenShingles = groupedFlattenShingles.collectAsMap()

In [43]:
val getDocID = 3;

var t1 = System.nanoTime

var LSH_Reslut_GetDoc = LSH_pairs(getDocID)

var duration = (System.nanoTime - t1) / 1e9d
println("LSH Duration: "+duration+" secs")

var t2 = System.nanoTime

val docShingle = spark.sparkContext.broadcast(mapFlattenShingles(getDocID).toVector)

var LinearResult = groupedFlattenShingles.map{
    case(id, buffer) =>
    var returnValue = -1
    if(id != getDocID){
      val compShingle = buffer.toVector 
      if(docShingle.value.intersect(compShingle).size.toDouble / docShingle.value.union(compShingle).size.toDouble >= 0.01){
          returnValue = id
      }
    }
    returnValue
}.filter(x => x!= -1).sortBy(x => x).collect()

var duration2 = (System.nanoTime - t1) / 1e9d
println("Linear Duration: "+duration2+" secs")

LSH Duration: 4.0371E-5 secs
Linear Duration: 1.331535873 secs


getDocID = 3
t1 = 4818461902939
LSH_Reslut_GetDoc = List(22, 26, 35, 50, 99, 168, 169, 197, 200, 214, 250, 277, 295, 301, 333, 351, 362, 366, 380, 388, 410, 427, 444, 466, 476, 490, 500, 518, 609, 662, 721, 729, 737, 798, 811, 860, 868, 875, 903)
duration = 4.0371E-5
t2 = 4818462818423
docShingle = Broadcast(53)
LinearResult = Array(24, 111, 141, 173, 177, 551, 611, 622, 735, 742, 743, 756, 789)
duration2 = 1.331535873


1.331535873

103820004 Michael Fu